In [131]:
import torch 
import torch.nn.functional as F 
import matplotlib.pyplot as plt
%matplotlib inline

In [132]:
words = open('names.txt', 'r').read().splitlines()
words[:3]

['emma', 'olivia', 'ava']

In [133]:
len(words)

32033

In [134]:
chars = sorted(list(set(''.join(words))))
stoi = {s:i+1 for i, s in enumerate(chars)}
stoi["."] = 0
itos = {i:s for s, i in stoi.items()}
itos

{1: 'a',
 2: 'b',
 3: 'c',
 4: 'd',
 5: 'e',
 6: 'f',
 7: 'g',
 8: 'h',
 9: 'i',
 10: 'j',
 11: 'k',
 12: 'l',
 13: 'm',
 14: 'n',
 15: 'o',
 16: 'p',
 17: 'q',
 18: 'r',
 19: 's',
 20: 't',
 21: 'u',
 22: 'v',
 23: 'w',
 24: 'x',
 25: 'y',
 26: 'z',
 0: '.'}

In [135]:
block_size = 3
X, Y = [], []

for w in words:
    context = [0] * block_size
    for ch in w + '.':
        ix = stoi[ch]
        X.append(context)
        Y.append(ix)
        context = context[1:] + [ix]

X = torch.tensor(X)
Y = torch.tensor(Y)

In [136]:
X.shape, X.dtype, Y.shape, Y.dtype

(torch.Size([228146, 3]), torch.int64, torch.Size([228146]), torch.int64)

In [137]:
def build_dataset(words):
    block_size = 5
    X, Y = [], []

    for w in words:
        context = [0] * block_size
        for ch in w + '.':
            ix = stoi[ch]
            X.append(context)
            Y.append(ix)
            context = context[1:] + [ix]
    X = torch.tensor(X)
    Y = torch.tensor(Y)
    print(X.shape, Y.shape)
    return X, Y

import random 
random.seed(42)
random.shuffle(words)
n1 = int(0.8*len(words))
n2 = int(0.9*len(words))

Xtr, Ytr = build_dataset(words[:n1])
Xdev, Ydev = build_dataset(words[n1:n2])
Xte, Yte = build_dataset(words[n2:])

torch.Size([182625, 5]) torch.Size([182625])
torch.Size([22655, 5]) torch.Size([22655])
torch.Size([22866, 5]) torch.Size([22866])


In [138]:
C = torch.randn((27, 2))

In [139]:
C[5]

tensor([ 0.0053, -0.4203])

In [140]:
emb = C[X]
emb.shape

torch.Size([228146, 3, 2])

In [141]:
W1 = torch.randn((6, 100))
b1 = torch.randn(100)

In [142]:
h = torch.tanh((emb.view(-1, 6) @ W1) + b1)

In [143]:
h

tensor([[-0.9117,  0.9973,  0.9999,  ..., -0.7351,  0.9950,  0.9996],
        [-0.7639,  0.9329,  0.9999,  ..., -0.2676,  0.9627,  0.9977],
        [-0.8904,  0.9997,  0.9998,  ..., -0.5294,  0.9993,  0.9981],
        ...,
        [ 0.7652,  0.6521,  0.9994,  ...,  0.9794,  0.9995, -0.8492],
        [ 0.8926, -0.4907,  0.9999,  ...,  0.9833,  0.9988, -0.6619],
        [ 0.9870, -0.9629,  0.9996,  ...,  0.9968,  0.7925, -0.7530]])

In [144]:
h.shape

torch.Size([228146, 100])

In [145]:
W2 = torch.randn((100, 27))
b2 = torch.randn(27)

In [146]:
logits = (h @ W2) + b2

In [147]:
logits.shape

torch.Size([228146, 27])

In [148]:
counts = logits.exp()
prob = counts / counts.sum(1, keepdim=True)

In [149]:
prob.shape

torch.Size([228146, 27])

In [150]:
loss = -prob[torch.arange(prob.shape[0]), Y].log().mean()

In [151]:
loss

tensor(16.6361)

In [182]:
block_size = 5
C = torch.randn((27, 10))
dim1 = block_size*C.shape[1]
W1 = torch.randn((dim1, 200))
b1 = torch.randn(200)
W2 = torch.randn((200, 27))
b2 = torch.randn(27)
parameters = [C, W1, W2, b1, b2]

In [183]:
sum(p.nelement() for p in parameters)

15897

In [184]:
for p in parameters:
    p.requires_grad = True


In [185]:
lre = torch.linspace(-3, 0, 1000)
lrs = 10**lre

In [186]:
lri = []
lossi = []
stepi = []

In [193]:
for i in range(200000):
    # making batches
    ix = torch.randint(0, Xtr.shape[0], (32,))
    # forward pass
    emb = C[Xtr[ix]]
    h = torch.tanh(emb.view(-1, dim1) @ W1 + b1)
    logits = h @ W2 + b2
    loss = F.cross_entropy(logits, Ytr[ix])
    # backwards pass
    for p in parameters:
        p.grad = None
    loss.backward()
    # update
    lr = 0.1 if i < 100000 else 0.01
    for p in parameters:
        p.data += -lr * p.grad
    stepi.append(i)
    lossi.append(loss.log10().item())

print(loss.item())

2.5886523723602295


In [194]:
emb = C[Xtr]
h = torch.tanh(emb.view(-1, dim1) @ W1 + b1)
logits = h @ W2 + b2
loss = F.cross_entropy(logits, Ytr)
loss

tensor(2.1279, grad_fn=<NllLossBackward0>)

In [195]:
emb = C[Xdev]
h = torch.tanh(emb.view(-1, dim1) @ W1 + b1)
logits = h @ W2 + b2
loss = F.cross_entropy(logits, Ydev)
loss

tensor(2.1634, grad_fn=<NllLossBackward0>)

In [196]:
# training split, dev/validation split, test split
# 80%, 10%, 10%

In [197]:
block_size = 5
for _ in range(20):
    out = []
    context = [0] * block_size

    while True:
        emb = C[torch.tensor([context])]
        h = torch.tanh(emb.view(1, -1) @ W1 + b1)
        logits = h @ W2 + b2
        probs = F.softmax(logits, dim=1)
        ix = torch.multinomial(probs, num_samples=1, replacement=True)
        context = context[1:] + [ix]
        out.append(itos[ix.item()])
        if ix == 0:
            print(''.join(out[:-1]))
            out = []
            break 

sheeh
raila
meri
jinthayan
samina
rylie
bamou
akley
yanelysan
joze
kendifari
korissi
adar
adislifryn
dhilak
hevistaliann
trisa
evelyn
praphten
kurhay
